In [2]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
#binarize the labels
lb = LabelBinarizer()

import pandas as pd

data_files = ['cle_train.csv','cle_test.csv','hun_train.csv','hun_test.csv','swi_train.csv','swi_test.csv','vir_train.csv','vir_test.csv']

datasets = {}

for file in data_files:
    data = pd.read_csv('../TrainTestData/' + file)
    
    X = data.iloc[:, :-1]
    Y = data.iloc[:, -1]
    
    Y_binary = Y.apply(lambda x: 1 if x > 0 else 0)
    
    # Extract the name from the file path
    name = file.split('.')[0]
    
    # Store the dataset components in a dictionary
    datasets[name] = {'X': X, 'Y': Y, 'Y_binary': Y_binary}

# Unpack the dictionary values in a loop
variables = ['cle', 'hun', 'swi', 'vir']
train_test = ['train', 'test']

for var in variables:
    for tt in train_test:
        X, Y, Y_binary = datasets[f'{var}_{tt}'].values()
        globals()[f'{var}_X_{tt}'] = X
        globals()[f'{var}_Y_{tt}'] = Y
        globals()[f'{var}_Y_{tt}_binary'] = Y_binary

In [4]:
X_test = pd.concat([cle_X_test,hun_X_test,swi_X_test,vir_X_test])
y_test = pd.concat([cle_Y_test_binary,hun_Y_test_binary,swi_Y_test_binary,vir_Y_test_binary])

X_train = pd.concat([cle_X_train,hun_X_train,swi_X_train,vir_X_train])
y_train = pd.concat([cle_Y_train_binary,hun_Y_train_binary,swi_Y_train_binary,vir_Y_train_binary])

In [5]:
def create_clients():
    cle_zip = list(zip(cle_X_train.values,cle_Y_train_binary))
    hun_zip = list(zip(hun_X_train.values,hun_Y_train_binary))
    vir_zip = list(zip(vir_X_train.values,vir_Y_train_binary))
    swi_zip = list(zip(swi_X_train.values,swi_Y_train_binary))
    
    shards = [cle_zip, hun_zip, vir_zip,swi_zip]
    client_names = ["client_1","client_2","client_3","client_4"]
    dic = {client_names[i] : shards[i] for i in range(len(client_names))}
    return dic


def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


class CNN:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(22,1)))
        model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model 

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    length = len(y_test)
    Y_test = tf.reshape(Y_test,(length,1))
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), Y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [10]:
cle_model = tf.keras.models.load_model('../Models/CNN_only/cle_cnn.h5')
vir_model = tf.keras.models.load_model('../Models/CNN_only/vir_cnn.h5')
hun_model = tf.keras.models.load_model('../Models/CNN_only/hun_cnn.h5')
swi_model = tf.keras.models.load_model('../Models/CNN_only/swi_cnn.h5')

In [42]:
global_weights_average = [(x+y+z+m)/4 for x,y,z,m in zip(cle_model.get_weights(),vir_model.get_weights(),hun_model.get_weights(),swi_model.get_weights())]
global_weights_average

[array([[[ 2.64239591e-02,  3.05008460e-02,  4.27427981e-03,
          -1.80557631e-02,  9.28547140e-03,  8.68122745e-03,
           5.94170764e-02,  3.66329402e-02, -1.85543336e-02,
          -9.56844538e-04, -1.44479508e-02, -7.20000081e-03,
           1.62690654e-02, -8.39515589e-03,  1.88019071e-02,
          -4.94330786e-02, -3.55329514e-02, -5.07773412e-03,
           1.37019781e-02, -2.74410360e-02, -1.02027133e-02,
           3.65221724e-02, -2.54375562e-02,  1.86088551e-02,
          -2.05711685e-02, -3.96512449e-02,  1.18771726e-02,
          -9.24333744e-03, -1.69381425e-02,  2.54502241e-02,
           2.75334623e-02,  4.91359383e-02, -7.14757014e-03,
           2.26549581e-02,  1.09521672e-04, -1.48015451e-02,
          -1.79945640e-02, -2.12781280e-02,  4.06436399e-02,
           2.63476707e-02,  3.11362483e-02,  1.61282381e-03,
          -4.74493988e-02,  3.04251425e-02, -6.23036316e-03,
           4.59343269e-02,  1.55795040e-02,  9.83753707e-03,
           4.53509856e-0

In [68]:
#create clients
clients = create_clients()

#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100
    
#create optimizer
lr = 0.01 
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(lr=lr, decay=lr / comms_round, momentum=0.9) 

#initialize global model
smlp_global = CNN()
global_model = smlp_global.build(22, 2)

#set the initial weights to average of cnn models
global_model.set_weights(global_weights_average) 

list_getweights = []
list_setweights =[]

#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights() 
    list_getweights.append(global_weights[0][0][0])
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = CNN()
        local_model = smlp_local.build(22, 2)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=5, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model 
    global_model.set_weights(average_weights)
    list_setweights.append(global_model.get_weights()[0][0][0])
    
    
    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(250)
        smlp_SGD = CNN()
        SGD_model = smlp_SGD.build(22, 2) 

        SGD_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

D:\Anaconda\envs\GPU\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - 0s 9ms/step
comm_round: 0 | global_acc: 55.596% | global_loss: 0.6923404932022095
9/9 [==============================] - 0s 9ms/step
comm_round: 1 | global_acc: 55.596% | global_loss: 0.6923796534538269
9/9 [==============================] - 0s 8ms/step
comm_round: 2 | global_acc: 55.596% | global_loss: 0.6919292211532593
9/9 [==============================] - 0s 8ms/step
comm_round: 3 | global_acc: 55.596% | global_loss: 0.6921492218971252
9/9 [==============================] - 0s 7ms/step
comm_round: 4 | global_acc: 55.596% | global_loss: 0.6919586658477783


KeyboardInterrupt: 

In [73]:
list_getweights[1] - list_setweights[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [6]:
Y_predictions = np.argmax(SGD_model.predict(X_test),axis = 1)

In [7]:
cm = confusion_matrix(Y_predictions, Y_test)
cm

array([[ 97,  15],
       [ 26, 139]], dtype=int64)

In [8]:
print(classification_report(Y_test, Y_predictions))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83       123
           1       0.84      0.90      0.87       154

    accuracy                           0.85       277
   macro avg       0.85      0.85      0.85       277
weighted avg       0.85      0.85      0.85       277



# Testing on each dataset

In [10]:
Y_cle = np.argmax(SGD_model.predict(cle_X_test),axis = 1)
cm_cle = confusion_matrix(Y_cle, cle_Y_test_binary)
print(cm_cle)
print(classification_report(Y_cle, cle_Y_test_binary))

[[42  3]
 [ 9 37]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87        45
           1       0.93      0.80      0.86        46

    accuracy                           0.87        91
   macro avg       0.87      0.87      0.87        91
weighted avg       0.87      0.87      0.87        91



In [11]:
Y_vir = np.argmax(SGD_model.predict(vir_X_test),axis = 1)
cm_vir = confusion_matrix(Y_vir, vir_Y_test_binary)
print(cm_vir)
print(classification_report(Y_vir, vir_Y_test_binary))

[[ 5  5]
 [ 8 42]]
              precision    recall  f1-score   support

           0       0.38      0.50      0.43        10
           1       0.89      0.84      0.87        50

    accuracy                           0.78        60
   macro avg       0.64      0.67      0.65        60
weighted avg       0.81      0.78      0.79        60



In [12]:
Y_hun = np.argmax(SGD_model.predict(hun_X_test),axis = 1)
cm_hun = confusion_matrix(Y_hun, hun_Y_test_binary)
print(cm_hun)
print(classification_report(Y_hun, hun_Y_test_binary))

[[49  4]
 [ 8 28]]
              precision    recall  f1-score   support

           0       0.86      0.92      0.89        53
           1       0.88      0.78      0.82        36

    accuracy                           0.87        89
   macro avg       0.87      0.85      0.86        89
weighted avg       0.87      0.87      0.86        89



In [13]:
Y_swi = np.argmax(SGD_model.predict(swi_X_test),axis = 1)
cm_swi = confusion_matrix(Y_swi, swi_Y_test_binary)
print(cm_swi)
print(classification_report(Y_swi, swi_Y_test_binary))

[[ 1  3]
 [ 1 32]]
              precision    recall  f1-score   support

           0       0.50      0.25      0.33         4
           1       0.91      0.97      0.94        33

    accuracy                           0.89        37
   macro avg       0.71      0.61      0.64        37
weighted avg       0.87      0.89      0.88        37



In [15]:
SGD_model.save('CNN-FL_85%.h5')

ValueError: Unable to create dataset (name already exists)